In [17]:
import pandas as pd
import pygrib
import numpy as np
import gdown

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
import zipfile
import os

from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [2]:
countries_zip_url = "https://drive.google.com/uc?id=1UQzdO7suT0BnwKBeNybMG97vM9GIDogA"
countries_zip_file_path = "../allCountries.zip"

# Download the ZIP file if it doesn't exist; otherwise, proceed to read the TXT file.
if not os.path.exists(countries_zip_file_path):
    gdown.download(countries_zip_url, countries_zip_file_path, quiet=False)

with zipfile.ZipFile(countries_zip_file_path) as z:
    countries_txt_filename = "allCountries.txt"

    with z.open(countries_txt_filename) as txt_file:
        countries_df = pd.read_csv(txt_file, sep="\t", header=None)
countries_df

/var/folders/9m/ym86jvl93wq3tx6ssy8ql7kh0000gn/T/ipykernel_27349/3155508545.py:12: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  countries_df = pd.read_csv(txt_file, sep="\t", header=None)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2994701,Roc Meler,Roc Meler,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74180,T,PK,AD,"AD,FR",02,NaN,NaN,NaN,0,2811.0,2348,Europe/Andorra,2023-10-03
1,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
2,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05
3,3023203,Port Vieux de la Coume d’Ose,Port Vieux de la Coume d'Ose,"Port Vieux de Coume d'Ose,Port Vieux de Coume ...",42.62568,1.61823,T,PASS,AD,NaN,00,NaN,NaN,NaN,0,NaN,2687,Europe/Andorra,2014-11-05
4,3029315,Port de la Cabanette,Port de la Cabanette,"Port de la Cabanette,Porteille de la Cabanette",42.60000,1.73333,T,PASS,AD,"AD,FR",B3,09,91.0,9139.0,0,NaN,2379,Europe/Andorra,2014-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950180,13051060,Kabaresy Seamount,Kabaresy Seamount,NaN,-6.30607,125.51837,U,SMU,NaN,NaN,00,NaN,NaN,NaN,0,NaN,-9999,NaN,2024-08-14
12950181,13051061,Menjangan Hill,Menjangan Hill,NaN,-8.06415,114.51922,U,HLLU,NaN,NaN,02,NaN,NaN,NaN,0,NaN,-9999,NaN,2024-08-14
12950182,13061112,Al Is‘āf wa al Maţāfi’,Al Is`af wa al Matafi',"Al Is`af wa al Matafi',Al Is‘āf wa al Maţāfi’,...",32.18985,34.97129,S,FIRE,NaN,NaN,02,NaN,NaN,NaN,0,NaN,98,Asia/Jerusalem,2024-09-13
12950183,13061113,Al Iţfā’īyah,Al Itfa'iyah,"Al Itfa'iyah,Al Iţfā’īyah,Fire Brigade,alatfay...",32.22261,35.26492,S,FIRE,NaN,NaN,WE,NaN,NaN,NaN,0,NaN,512,NaN,2024-09-13


In [3]:
eui_url = "https://drive.google.com/uc?id=12qGq_DLefI1RihIF_RKQUyJtm480-xRC"
eui_df = pd.read_csv(eui_url)
eui_df

,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year)
0,Nha Trang,1572151,Vietnam,59.096065,112.778867
1,Aberdeen,2657832,United Kingdom,231.302877,259.832393
2,Abidjan,2293538,Cote d'Ivoire,73.830819,105.622137
3,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457
4,Abuja,2352778,Nigeria,63.955819,103.009079
...,...,...,...,...,...
477,Yerevan,616052,Armenia,198.865302,132.124738
478,Yokohama,1848354,Japan,167.095118,228.612531
479,Zagreb,3186886,Croatia,92.676724,120.543989
480,Zanzibar,148730,Tanzania,65.015086,104.746247


In [4]:
merged_df = pd.merge(
    countries_df, eui_df, left_on=0, right_on="Geonames ID", how="inner"
)
merged_df.rename(columns={4: "latitud", 5: "longitud"}, inplace=True)
merged_df

,0,1,2,3,latitud,longitud,6,7,8,9,...,14,15,16,17,18,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year)
0,292968,Abu Dhabi,Abu Dhabi,"A-pu-that-pi,AEbu Saby,AUH,Aboe Dhabi,Abou Dab...",24.45118,54.39696,P,PPLC,AE,NaN,...,1807000,NaN,6,Asia/Dubai,2024-03-27,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457
1,1138958,Kabul,Kabul,"Cabool,Caboul,Cabul,Cabura,Cabúl,Caubul,KBL,Ka...",34.52813,69.17233,P,PPLC,AF,NaN,...,4434550,NaN,1798,Asia/Kabul,2024-09-05,Kabul,1138958,Afghanistan,213.167026,144.395840
2,3183875,Tirana,Tirana,"TIA,Terana,Theranda,Tiorana,Tiorána,Tiran,Tira...",41.32750,19.81889,P,PPLC,AL,NaN,...,418495,NaN,113,Europe/Tirane,2023-01-01,Tirana,3183875,Albania,133.717672,101.873579
3,616052,Yerevan,Yerevan,"Ayrivan,Djerevan,EVN,Eireavan,Eireaván,Ereban,...",40.18111,44.51361,P,PPLC,AM,NaN,...,1093485,NaN,994,Asia/Yerevan,2023-11-13,Yerevan,616052,Armenia,198.865302,132.124738
4,2240449,Luanda,Luanda,"LAD,Loanda,Louanda,Louanta,Luand,Luanda,Luanda...",-8.83682,13.23432,P,PPLC,AO,NaN,...,2776168,NaN,73,Africa/Luanda,2024-03-26,Luanda,2240449,Angola,65.343750,104.311100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1018725,Bloemfontein,Bloemfontein,"BFN,Bloemfontein,Blumfantehjn,Blumfonteina,Blu...",-29.12107,26.21400,P,PPLA,ZA,NaN,...,556000,NaN,1396,Africa/Johannesburg,2022-08-16,Bloemfontein,1018725,South Africa,78.735991,106.046441
478,3369157,Cape Town,Cape Town,"Altepetl In Cabo,Ar Chab,CPT,Cape Toun,Cape To...",-33.92584,18.42322,P,PPLA,ZA,NaN,...,4710000,NaN,25,Africa/Johannesburg,2024-03-27,Cape Town,3369157,South Africa,71.455819,104.301427
479,909137,Lusaka,Lusaka,"LUN,Lousaka,Louzaka,Lusaca,Lusak,Lusaka,Lusako...",-15.40669,28.28713,P,PPLC,ZM,NaN,...,1267440,NaN,1277,Africa/Lusaka,2019-09-05,Lusaka,909137,Zambia,75.533405,105.500787
480,890299,Harare,Harare,"Arare,Charare,HRE,Harare,Hararensis Urbs,Harar...",-17.82772,31.05337,P,PPLC,ZW,NaN,...,1542813,NaN,1494,Africa/Harare,2019-09-05,Harare,890299,Zimbabwe,69.613147,102.108799


In [5]:
grib_url = 'https://drive.google.com/uc?id=1gqE5yyxQToh0ztY9g1PbH1odL3YKzAYh'
grib_file_path = "../data.grib"


gdown.download(grib_url, grib_file_path, quiet=False)

grbs = pygrib.open(grib_file_path)
grib_message = grbs[1]
print(grib_message)
grbs.close()

Downloading...
From: https://drive.google.com/uc?id=1gqE5yyxQToh0ztY9g1PbH1odL3YKzAYh
To: /Users/barbaraflores/Desktop/MIDS/IDS798_Capstone/data.grib
100%|██████████| 2.08M/2.08M [00:00<00:00, 17.6MB/s]


1:Skin temperature:K (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 202408221800


In [6]:
def get_nearest_temperature(latitude, longitude, grib_message):
    # Retrieve the complete data
    values, lats, lons = grib_message.data()

    # Calculate the distance
    distances = np.sqrt((lats - latitude) ** 2 + (lons - longitude) ** 2)
    idx_min = np.unravel_index(np.argmin(distances), distances.shape)

    # Get the temperature at the nearest coordinate
    nearest_temperature = values[idx_min]

    return nearest_temperature

In [7]:
def test_get_nearest_temperature(latitude, longitude, expected_temperature):
    tolerance = 1e-4
    actual_temperature = get_nearest_temperature(latitude, longitude, grib_message)

    if abs(actual_temperature - expected_temperature) <= tolerance:
        print("Barbara's code works!")
    else:
        print("do something!")


latitude = 9.0
longitude = 38.75
expected_temperature = 288.161743
test_get_nearest_temperature(latitude, longitude, expected_temperature)

latitude = -9.5
longitude = 38.50
expected_temperature = 300.204712
test_get_nearest_temperature(latitude, longitude, expected_temperature)

latitude = 34.0
longitude = 9.00
expected_temperature = 305.109009
test_get_nearest_temperature(latitude, longitude, expected_temperature)

do something!
do something!
do something!


In [8]:
# test data from perfect merged scenario
test_data = {
    "City": ["Addis Ababa", "Lindi", "Tunis"],
    "Latitude": [9.0, -9.5, 34.0],
    "Longitude": [38.75, 38.50, 9.00],
    "Temperature": [288.161743, 300.204712, 305.109009],
}
test_data = pd.DataFrame(test_data)

In [9]:
merged_df["temperature"] = merged_df.apply(
    lambda row: get_nearest_temperature(row["latitud"], row["longitud"], grib_message),
    axis=1,
)
merged_df

,0,1,2,3,latitud,longitud,6,7,8,9,...,15,16,17,18,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year),temperature
0,292968,Abu Dhabi,Abu Dhabi,"A-pu-that-pi,AEbu Saby,AUH,Aboe Dhabi,Abou Dab...",24.45118,54.39696,P,PPLC,AE,NaN,...,NaN,6,Asia/Dubai,2024-03-27,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457,307.920670
1,1138958,Kabul,Kabul,"Cabool,Caboul,Cabul,Cabura,Cabúl,Caubul,KBL,Ka...",34.52813,69.17233,P,PPLC,AF,NaN,...,NaN,1798,Asia/Kabul,2024-09-05,Kabul,1138958,Afghanistan,213.167026,144.395840,287.799576
2,3183875,Tirana,Tirana,"TIA,Terana,Theranda,Tiorana,Tiorána,Tiran,Tira...",41.32750,19.81889,P,PPLC,AL,NaN,...,NaN,113,Europe/Tirane,2023-01-01,Tirana,3183875,Albania,133.717672,101.873579,299.332779
3,616052,Yerevan,Yerevan,"Ayrivan,Djerevan,EVN,Eireavan,Eireaván,Ereban,...",40.18111,44.51361,P,PPLC,AM,NaN,...,NaN,994,Asia/Yerevan,2023-11-13,Yerevan,616052,Armenia,198.865302,132.124738,295.756607
4,2240449,Luanda,Luanda,"LAD,Loanda,Louanda,Louanta,Luand,Luanda,Luanda...",-8.83682,13.23432,P,PPLC,AO,NaN,...,NaN,73,Africa/Luanda,2024-03-26,Luanda,2240449,Angola,65.343750,104.311100,294.705826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,1018725,Bloemfontein,Bloemfontein,"BFN,Bloemfontein,Blumfantehjn,Blumfonteina,Blu...",-29.12107,26.21400,P,PPLA,ZA,NaN,...,NaN,1396,Africa/Johannesburg,2022-08-16,Bloemfontein,1018725,South Africa,78.735991,106.046441,283.180435
478,3369157,Cape Town,Cape Town,"Altepetl In Cabo,Ar Chab,CPT,Cape Toun,Cape To...",-33.92584,18.42322,P,PPLA,ZA,NaN,...,NaN,25,Africa/Johannesburg,2024-03-27,Cape Town,3369157,South Africa,71.455819,104.301427,284.783951
479,909137,Lusaka,Lusaka,"LUN,Lousaka,Louzaka,Lusaca,Lusak,Lusaka,Lusako...",-15.40669,28.28713,P,PPLC,ZM,NaN,...,NaN,1277,Africa/Lusaka,2019-09-05,Lusaka,909137,Zambia,75.533405,105.500787,291.205826
480,890299,Harare,Harare,"Arare,Charare,HRE,Harare,Hararensis Urbs,Harar...",-17.82772,31.05337,P,PPLC,ZW,NaN,...,NaN,1494,Africa/Harare,2019-09-05,Harare,890299,Zimbabwe,69.613147,102.108799,289.385513


In [25]:
X_knn = merged_df[["latitud", "longitud"]]
y_knn = merged_df["Residential EUI (kWh/m2/year)"]

X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(
    X_knn, y_knn, test_size=0.2, random_state=42
)

knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_knn, y_train_knn)

y_pred_knn = knn_model.predict(X_test_knn)

mse_knn = mean_squared_error(y_test_knn, y_pred_knn)
r2_knn = r2_score(y_test_knn, y_pred_knn)
mae_knn = mean_absolute_error(y_test_knn, y_pred_knn)
rmse_knn = np.sqrt(mse_knn)
mape_knn = mean_absolute_percentage_error(y_test_knn, y_pred_knn)
wape_knn = np.sum(np.abs(y_test_knn - y_pred_knn)) / np.sum(np.abs(y_test_knn)) * 100


print("KNN - MSE: {:.3f}".format(mse_knn))
print("KNN - R²: {:.3f}".format(r2_knn))
print("KNN - MAE: {:.3f}".format(mae_knn))
print("KNN - RMSE: {:.3f}".format(rmse_knn))
print("KNN - MAPE: {:.3f}".format(mape_knn))
print("KNN - WAPE: {:.3f}%".format(wape_knn))

KNN - MSE: 874.987
KNN - R²: 0.792
KNN - MAE: 18.912
KNN - RMSE: 29.580
KNN - MAPE: 17.524
KNN - WAPE: 18.339%


In [24]:
X_linear = merged_df[["temperature"]]
y_linear = merged_df["Residential EUI (kWh/m2/year)"]

X_train_linear, X_test_linear, y_train_linear, y_test_linear = train_test_split(
    X_linear, y_linear, test_size=0.2, random_state=42
)

linear_model = LinearRegression()
linear_model.fit(X_train_linear, y_train_linear)

y_pred_linear = linear_model.predict(X_test_linear)

mse_linear = mean_squared_error(y_test_linear, y_pred_linear)
r2_linear = r2_score(y_test_linear, y_pred_linear)
mae_linear = mean_absolute_error(y_test_linear, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)
mape_linear = mean_absolute_percentage_error(y_test_linear, y_pred_linear)
wape_linear = np.sum(np.abs(y_test_linear - y_pred_linear)) / np.sum(np.abs(y_test_linear)) * 100

 
print("Linear Regression - MSE: {:.3f}".format(mse_linear))
print("Linear Regression - R²: {:.3f}".format(r2_linear))
print("Linear Regression - MAE: {:.3f}".format(mae_linear))
print("Linear Regression - RMSE: {:.3f}".format(rmse_linear))
print("Linear Regression - MAPE: {:.3f}".format(mape_linear))
print("Linear Regression - WAPE: {:.3f}%".format(wape_linear))

Linear Regression - MSE: 3942.661
Linear Regression - R²: 0.065
Linear Regression - MAE: 52.761
Linear Regression - RMSE: 62.791
Linear Regression - MAPE: 61.465
Linear Regression - WAPE: 51.161%
